# 04 ポップコーン

### メモ  
GAOZで完全一致しないやつがある  
->ハミング距離最小のやつで絞る

### 担当者と日付
2024/10/1 渡邉健  
2024/10/2 ごとう

## 準備

### ライブラリのインポート  

In [98]:
import pandas as pd
import numpy as np
import warnings #ヘッダとかのやつが邪魔だったのでファイルにまとめてインポートしてる
warnings.simplefilter(action='ignore', category=FutureWarning)

In [99]:
# fmt: off
ALL_HEADER = ['Name', 'Gender', 'Age', 'Occupation', 'ZIP-code', '2', '56', '247', '260', '653', '673', '810', '885',  # noqa: E501
                      '1009', '1073', '1097', '1126', '1525', '1654', '1702', '1750', '1881', '1920', '1967', '2017',  # noqa: E501
                      '2021', '2043', '2086', '2087', '2093', '2100', '2105', '2138', '2143', '2174', '2193', '2253',  # noqa: E501
                      '2399', '2628', '2797', '2872', '2968', '3393', '3438', '3439', '3440', '3466', '3479', '3489',  # noqa: E501
                      '3877', '3889'] # noqa

B_HEADERS_LIST = [
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '260', '653', '1525', '2105', '2193', '2253', '2628', '2872', '3438', '3439', '3440', '3877', '3889'],  # noqa: E501
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '2', '56', '260', '653', '673', '1009', '1073', '1525', '1750', '1881', '1967', '2043', '2093', '2105', '2143', '2193', '2399', '2628', '2968', '3479', '3489', '3877', '3889'],  # noqa: E501
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '673', '1881', '1920', '2087', '2138'],  # noqa: E501
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '2', '56', '673', '810', '885', '1009', '1073', '1097', '1525', '1654', '1702', '1750', '1881', '1920', '1967', '2017', '2043', '2087', '2093', '2138', '2399', '3438', '3439', '3440'],  # noqa: E501
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '673', '810', '1073', '1126', '1702', '2100', '2174', '2253', '2797', '3393', '3466'],  # noqa: E501
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '247', '885', '1097', '1654', '2086', '2138', '2872'],  # noqa: E501
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '247', '2100', '2143', '2872', '3479'],  # noqa: E501
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '260', '1097', '1750', '2021', '2093', '2105', '2628', '2968'],  # noqa: E501
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '247', '1920', '2017', '2087'],  # noqa: E501
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '260', '1097', '2628', '2174', '2797', '1073', '2100', '2968', '2105', '2193'],  # noqa: E501
]

B_USER_ATTRIBUTE_HEADERS = ['Gender', 'Age', 'Occupation', 'ZIP-code']

B_REVIEW_HEADERS_LIST = [
  ['260', '653', '1525', '2105', '2193', '2253', '2628', '2872', '3438', '3439', '3440', '3877', '3889'],  # noqa: E501
  ['2', '56', '260', '653', '673', '1009', '1073', '1525', '1750', '1881', '1967', '2043', '2093', '2105', '2143', '2193', '2399', '2628', '2968', '3479', '3489', '3877', '3889'],  # noqa: E501
  ['673', '1881', '1920', '2087', '2138'],  # noqa: E501
  ['2', '56', '673', '810', '885', '1009', '1073', '1097', '1525', '1654', '1702', '1750', '1881', '1920', '1967', '2017', '2043', '2087', '2093', '2138', '2399', '3438', '3439', '3440'],  # noqa: E501
  ['673', '810', '1073', '1126', '1702', '2100', '2174', '2253', '2797', '3393', '3466'],  # noqa: E501
  ['247', '885', '1097', '1654', '2086', '2138', '2872'],  # noqa: E501
  ['247', '2100', '2143', '2872', '3479'],  # noqa: E501
  ['260', '1097', '1750', '2021', '2093', '2105', '2628', '2968'],  # noqa: E501
  ['247', '1920', '2017', '2087'],  # noqa: E501
  ['260', '1097', '2628', '2174', '2797', '1073', '2100', '2968', '2105', '2193'],  # noqa: E501
]

MOVIE_IDS = ['2', '56', '247', '260', '653', '673', '810', '885', '1009', '1073', '1097', '1126', '1525', '1654', '1702', '1750', '1881', '1920', '1967', '2017', '2021', '2043', '2086', '2087', '2093', '2100', '2105', '2138', '2143', '2174', '2193', '2253', '2399', '2628', '2797', '2872', '2968', '3393', '3438', '3439', '3440', '3466', '3479', '3489', '3877', '3889']
# fmt: on

### C_0 ~ C_9までの読み込み

In [100]:
from os import listdir

ATTACK_TARGET = "04"

#c0~c9に対するdfのリスト
c_data_list = []

files = listdir("../data")

for i in range(10):
    file = None
    for f in files:
        if f.startswith(f"C{ATTACK_TARGET}_{i}"):
            file = f
            break
    if file is None:
        print(f)
        raise("File not found")
    c_data = pd.read_csv(f"../data/C{ATTACK_TARGET}_{i}.csv")
    c_data_list.append(c_data)

c_data_list[0]

,Gender,Age,Occupation,ZIP-code,260,653,1525,2105,2193,2253,2628,2872,3438,3439,3440,3877,3889
0,F,56,6,977,3,0,2,4,0,4,2,3,0,1,0,2,0
1,M,18,19,934,5,3,0,0,4,3,1,2,2,0,0,5,3
2,M,35,14,794,0,3,2,4,4,0,0,5,3,1,4,1,0
3,F,56,0,550,3,2,1,4,1,1,3,5,0,4,4,4,1
4,M,50,10,186,1,5,0,0,3,2,5,1,3,5,2,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,M,18,17,284,2,2,0,4,4,0,5,2,1,0,0,1,4
9996,F,18,16,503,2,3,0,2,1,0,1,0,5,2,3,3,2
9997,M,35,3,494,4,3,0,0,0,4,3,0,2,0,1,3,0
9998,F,18,17,128,4,2,0,1,4,4,3,5,0,2,0,4,0


### Ba, Bbの読み込み

In [101]:
Ba = pd.read_csv(f"../data/B{ATTACK_TARGET}a.csv", names=ALL_HEADER)
Ba = Ba.drop(0)
Ba.head()

,Name,Gender,Age,Occupation,ZIP-code,2,56,247,260,653,...,2968,3393,3438,3439,3440,3466,3479,3489,3877,3889
1,Ximenes Huxtable,M,1,10,296,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Catlaina Dobby,F,35,1,919,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Audi Steggals,F,18,12,371,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Yuma Methley,M,25,17,224,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Rafael Cubley,M,1,1,495,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [102]:
Bb = pd.read_csv(f"../data/B{ATTACK_TARGET}b.csv")
Bb.head()

,2,56,247,260,653,673,810,885,1009,1073,...,2968,3393,3438,3439,3440,3466,3479,3489,3877,3889
0,1,4,4,2,0,5,3,1,1,*,...,0,0,3,0,2,3,3,3,0,1
1,3,3,4,1,4,4,4,4,4,1,...,0,4,0,4,2,4,0,4,2,2
2,2,2,0,5,3,0,0,0,0,4,...,0,0,4,0,0,0,4,0,0,0
3,1,2,0,3,0,0,0,5,*,1,...,0,2,5,4,2,4,0,2,1,0
4,5,4,1,4,3,5,4,1,3,3,...,2,0,2,5,4,4,1,0,1,2


## 分析

### GAOZに関して、BaのN番目の行と完全一致する行をC0~C9から取得　　
-GAOZへの変更が行われているか

In [103]:
# 試しにBaの3行目を選択
N = 3

gender = Ba.iloc[N]["Gender"]
age = int(Ba.iloc[N]["Age"])
occupation = int(Ba.iloc[N]["Occupation"])
zipcode = int(Ba.iloc[N]["ZIP-code"])

print("検索条件")
print(gender, age, occupation, zipcode)

filtered_C_list = []


検索条件
M 25 17 224


### GAOZが完全一致するものでなく、各Ciの中でHamming distが最小のものをリスト化する

In [104]:
print(c_data_list[0].iloc[1].iloc[0])
print(c_data_list[0].iloc[1].iloc[1])
print(c_data_list[0].iloc[1].iloc[2])
print(c_data_list[0].iloc[1].iloc[3])

M
18
19
934


In [105]:
import copy

#あるrowのhamming dist
base = [gender, age, occupation, zipcode]
# print(base)
def GAOZ_weighted_hamming(comp):
    ham = 0
    if comp['Gender'] != gender:
        ham += 1
    if comp['Age'] != age:
        ham += 2
    if  comp['Occupation'] != occupation:
        ham += 2
    if comp['ZIP-code'] != zipcode:
        ham += 4
    return ham

filtered_C_list = []
for c_index, cf in enumerate(c_data_list):
    min_hamming_distance = float("inf")
    cand = []
    # Cの10000レコードのそれぞれにハミング距離を計算
    for index, c_row in cf.iterrows():
        hamming_distance = GAOZ_weighted_hamming(c_row)
        if hamming_distance == min_hamming_distance:
            cand.append((c_row, hamming_distance))
        if hamming_distance < min_hamming_distance:
            min_hamming_distance = hamming_distance
            cand = [(c_row, hamming_distance)]
    # ハミング距離とCのファイルのindexを追加
    for elem, hamming_distance in cand:
        headers = copy.deepcopy(ALL_HEADER)
        headers.append("GAOZ_Hamming_Dist")
        headers.append("C_index")
        elem["GAOZ_Hamming_Dist"] = hamming_distance
        elem["C_index"] = c_index
        filtered_C_list.append(pd.Series(elem, index=headers))

final_df = pd.DataFrame(filtered_C_list)
final_df

,Name,Gender,Age,Occupation,ZIP-code,2,56,247,260,653,...,3438,3439,3440,3466,3479,3489,3877,3889,GAOZ_Hamming_Dist,C_index
1224,NaN,M,25,17,224,NaN,NaN,NaN,0.0,1.0,...,0.0,0.0,5.0,NaN,NaN,NaN,0.0,5.0,0,0
2457,NaN,M,50,17,224,5.0,0.0,NaN,0.0,2.0,...,NaN,NaN,NaN,NaN,5.0,2.0,1.0,0.0,2,1
4212,NaN,M,25,18,224,0.0,3.0,NaN,1.0,3.0,...,NaN,NaN,NaN,NaN,0.0,5.0,3.0,0.0,2,1
7736,NaN,M,25,8,224,2.0,0.0,NaN,0.0,3.0,...,NaN,NaN,NaN,NaN,0.0,5.0,0.0,0.0,2,1
6641,NaN,M,25,17,224,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2
8611,NaN,M,25,17,224,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2
737,NaN,M,25,6,224,0.0,0.0,NaN,NaN,NaN,...,2.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,2,3
1224,NaN,M,25,16,224,1.0,4.0,NaN,NaN,NaN,...,3.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,2,3
333,NaN,M,25,14,224,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,2,4
4311,NaN,M,25,9,224,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,2,4


### 各カラムごとに多数決の候補を表示する

In [106]:
# merged_rows = []

# for i, filtered_df in enumerate(filtered_C_list):
#     if not filtered_df.empty:
#         # 各フィルタリングされたデータフレームから1行だけ取得
#         row = filtered_df.iloc[0]
        
#         # ALL_HEADERに基づいて新しい行を作成
#         new_row = pd.Series(index=ALL_HEADER)
        
#         # データを転送
#         for col in ALL_HEADER:
#             if col in final_df.columns:
#                 print(col)
#                 new_row[col] = row[col]
#             else:
#                 new_row[col] = np.nan
        
#         # 新しい行をリストに追加
#         merged_rows.append(new_row)

# # リストからデータフレームを作成
# merged_df = pd.DataFrame(merged_rows, columns=ALL_HEADER)
# merged_df

Name


TypeError: 'float' object is not subscriptable

### データの再現　　
各行に対してランダムに一つ値を取得してみる

In [91]:
def select_random_value(column):
    # NaNを除外した値のリストを作成
    valid_values = column.dropna().tolist()
    if valid_values:
        # 有効な値がある場合、ランダムに1つ選択
        return np.random.choice(valid_values)
    else:
        # 有効な値がない場合はNaNを返す
        return np.nan

# 各カラムからランダムに値を選択
random_values = {}
for column in merged_df.columns:
    random_values[column] = select_random_value(merged_df[column])

# 結果を新しいデータフレームとして作成
result_df = pd.DataFrame([random_values])
result_df

,Name,Gender,Age,Occupation,ZIP-code,2,56,247,260,653,...,2968,3393,3438,3439,3440,3466,3479,3489,3877,3889
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 攻撃

### 以上の処理をBaの各行（0~49行）に対して実行する

In [93]:
import pandas as pd
import numpy as np

# final_df = pd.DataFrame(columns=cons.ALL_HEADER)

for row_index in range(50):
    # Baデータフレームから現在の行の情報を取得
    gender = Ba.iloc[row_index]["Gender"]
    age = Ba.iloc[row_index]["Age"]
    occupation = Ba.iloc[row_index]["Occupation"]
    zipcode = Ba.iloc[row_index]["ZIP-code"]

    filtered_C_list = []

    # 各cデータフレームに対して、完全一致するレコードをフィルタリング
    for c in c_data_list:
        filtered_C_0 = c[(c['Gender'] == gender) & 
                        (c['Age'] == int(age)) & 
                        (c['Occupation'] == int(occupation)) & 
                        (c['ZIP-code'] == int(zipcode))]
        filtered_C_list.append(filtered_C_0)

    # 多数決の候補となる行のリストを作成
    candidate_rows = []
    for i, filtered_df in enumerate(filtered_C_list):
        if not filtered_df.empty:
            # 各フィルタリングされたデータフレームから1行だけ取得
            row = filtered_df.iloc[0]
            # ALL_HEADERに基づいて新しい行を作成し、データを転送
            candidate_row = pd.Series(index=ALL_HEADER)
            for col in ALL_HEADER:
                candidate_row[col] = row[col] if col in filtered_df.columns else np.nan
            # 新しい行を候補リストに追加
            candidate_rows.append(candidate_row)

    # 候補行のリストからデータフレームを作成
    candidate_rows_df = pd.DataFrame(candidate_rows, columns=ALL_HEADER)

    # 各列に対して、ランダムに値を選択（多数決の実装）
    elected_value = {}
    for column in ALL_HEADER:
        elected_value[column] = select_random_value(candidate_rows_df[column])

    # 選択された値から新しい行を作成
    elected_value_row_df = pd.DataFrame([elected_value])

    # 選択された行をfinal_dfの最後に追加
    final_df = pd.concat([final_df, elected_value_row_df], ignore_index=True)

final_df

,Name,Gender,Age,Occupation,ZIP-code,2,56,247,260,653,...,3438,3439,3440,3466,3479,3489,3877,3889,GAOZ_Hamming_Dist,C_index
0,NaN,M,25.0,17.0,224.0,NaN,NaN,NaN,0.0,1.0,...,0.0,0.0,5.0,NaN,NaN,NaN,0.0,5.0,0.0,0.0
1,NaN,M,50.0,17.0,224.0,5.0,0.0,NaN,0.0,2.0,...,NaN,NaN,NaN,NaN,5.0,2.0,1.0,0.0,2.0,1.0
2,NaN,M,25.0,18.0,224.0,0.0,3.0,NaN,1.0,3.0,...,NaN,NaN,NaN,NaN,0.0,5.0,3.0,0.0,2.0,1.0
3,NaN,M,25.0,8.0,224.0,2.0,0.0,NaN,0.0,3.0,...,NaN,NaN,NaN,NaN,0.0,5.0,0.0,0.0,2.0,1.0
4,NaN,M,25.0,17.0,224.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,NaN,F,56.0,19.0,655.0,NaN,NaN,NaN,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
72,NaN,M,35.0,3.0,532.0,1.0,3.0,2.0,NaN,NaN,...,2.0,4.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73,NaN,F,35.0,5.0,110.0,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
74,NaN,M,45.0,4.0,988.0,NaN,NaN,5.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 最終的に得た映画のレビューの候補とBbを比較し、Bbの何番目のindexが一番近くてハミング距離にどれくらいの違いがあるかを取得する

In [97]:

def hamming_distance(a, b):
    a_int = a.fillna(0).astype(int)
    b_int = b.replace('*', None).fillna(0).astype(int)
    # ハミング距離の計算
    return np.sum(a_int != b_int)

def find_closest_row(row, Bb):
    distances = Bb.apply(lambda bb_row: hamming_distance(row[MOVIE_IDS], bb_row[MOVIE_IDS]), axis=1)
    min_distance = distances.min()
    closest_index = distances.idxmin()
    return pd.Series({'Min_Hamming_Distance': min_distance, 'Closest_Bb_Index': closest_index})

# 結果を格納するデータフレーム
result_df = pd.DataFrame(index=final_df.index, columns=['Min_Hamming_Distance', 'Closest_Bb_Index'])

# 各行に対してハミング距離を計算し、最も近い行を見つける
for idx, row in final_df[MOVIE_IDS].iterrows():
    result = find_closest_row(row, Bb[MOVIE_IDS])
    result_df.loc[idx] = result

# 元のfinal_dfにハミング距離の結果を結合
final_result_df = pd.concat([final_df, result_df], axis=1)
final_result_df

,Name,Gender,Age,Occupation,ZIP-code,2,56,247,260,653,...,3440,3466,3479,3489,3877,3889,GAOZ_Hamming_Dist,C_index,Min_Hamming_Distance,Closest_Bb_Index
0,NaN,M,25.0,17.0,224.0,NaN,NaN,NaN,0.0,1.0,...,5.0,NaN,NaN,NaN,0.0,5.0,0.0,0.0,11,35
1,NaN,M,50.0,17.0,224.0,5.0,0.0,NaN,0.0,2.0,...,NaN,NaN,5.0,2.0,1.0,0.0,2.0,1.0,18,45
2,NaN,M,25.0,18.0,224.0,0.0,3.0,NaN,1.0,3.0,...,NaN,NaN,0.0,5.0,3.0,0.0,2.0,1.0,20,35
3,NaN,M,25.0,8.0,224.0,2.0,0.0,NaN,0.0,3.0,...,NaN,NaN,0.0,5.0,0.0,0.0,2.0,1.0,16,45
4,NaN,M,25.0,17.0,224.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2.0,9,45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,NaN,F,56.0,19.0,655.0,NaN,NaN,NaN,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,35
72,NaN,M,35.0,3.0,532.0,1.0,3.0,2.0,NaN,NaN,...,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19,35
73,NaN,F,35.0,5.0,110.0,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,7,35
74,NaN,M,45.0,4.0,988.0,NaN,NaN,5.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,45
